In [1]:
import numpy as np
from scipy.io import loadmat


SHAUNAK BHIDE

In [2]:
results = loadmat('yalefaces.mat')
X = results['X']

In [3]:
#a
#grab first 10 faces
#Create labels for all photos
X_10 = X[:, :640]
y = np.repeat(range(10), 64)


In [4]:
# Part B: pick 14 scenes from each face to use as test data, and 50 scenes to use as training data
#Doing this for every face.
np.random.seed(70)
lighting_scenes = 64
faces_to_extract = 10
test_scenes = 14
train_scenes = 50

# Shuffle lighting scenes within each face
for i in range(faces_to_extract):
    face_start = i * lighting_scenes
    face_end = (i + 1) * lighting_scenes
    np.random.shuffle(X_10[:, face_start:face_end])

# Initialize training and test matrices
train_data = np.zeros((1024, faces_to_extract * train_scenes))
test_data = np.zeros((1024, faces_to_extract * test_scenes))

# Split the data into training and test sets
for i in range(faces_to_extract):
    face_start = i * lighting_scenes
    train_start = i * train_scenes
    test_start = i * test_scenes

    train_data[:, train_start:train_start + train_scenes] = X_10[:, face_start:face_start + train_scenes]
    test_data[:, test_start:test_start + test_scenes] = X_10[:, face_start + train_scenes:face_start + lighting_scenes]

In [5]:
# Part C:
from sklearn.decomposition import PCA
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.metrics import accuracy_score

# Perform PCA on the training data
n_modes = 20
pca = PCA(n_components=n_modes)
pca_train_data = pca.fit_transform(train_data.T)

# Create training labels
train_labels = np.repeat(np.arange(faces_to_extract), train_scenes)

# Perform LDA on the reduced training data
lda = LinearDiscriminantAnalysis()
lda.fit(pca_train_data, train_labels)

# Transform the test data using the same PCA model
pca_test_data = pca.transform(test_data.T)

# Create test labels
test_labels = np.repeat(np.arange(faces_to_extract), test_scenes)

# Make predictions on the reduced test data
predictions = lda.predict(pca_test_data)

# Calculate classification accuracy
accuracy = accuracy_score(test_labels, predictions)
print(f"Classification accuracy: {accuracy * 100:.2f}%")

Classification accuracy: 64.29%


In [8]:
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
# Part D: 
# Classifying faces using an SVM and a decision tree
# SVM classifier

# Transform the test data using the same PCA model
pca_test_data = pca.transform(test_data.T)

svm = SVC()
svm.fit(pca_train_data, train_labels)
y_pred_svm = svm.predict(pca_test_data)
accuracy_svm = accuracy_score(test_labels, y_pred_svm)
print(f"SVM Classification accuracy: {accuracy_svm * 100:.2f}%")

# decision tree classifier
dtc = DecisionTreeClassifier()
dtc.fit(pca_train_data, train_labels)
y_pred_dtc = dtc.predict(pca_test_data)
accuracy_dtc = accuracy_score(test_labels, y_pred_dtc)
print(f"Decision Tree Classification accuracy: {accuracy_dtc * 100:.2f}%")

SVM Classification accuracy: 82.86%
Decision Tree Classification accuracy: 87.14%


The SVM and decision tree were both very close, and able to classify the different faces the best, followed by the the LDA. Random seeds for the SVM and decision tree do affect the accuracy. The main reason for this is that we are randomly picking lighting scenes for each face and putting them all into the same datasets, so it is harder to linearly separate the data. 

SVM is a powerful classification algorithm that works by finding the best hyperplane that separates the data points of different classes in the feature space. SVM can handle high-dimensional data well and is robust against overfitting. In this case the SVM model was able to find a good separation boundary between the different faces in the dataset. The SVM is also able to handle nonlinearly seperable data.

Decision trees, on the other hand, are a simple and interpretable classification method that works by recursively splitting the data based on the feature that provides the most information gain. While decision trees can handle non-linearly separable data and can be easily visualized, they are prone to overfitting and can become very complex. In this case, the decision tree model achieved a good result.

 LDA works well when the classes are well-separated and the data is normally distributed. However, it may perform poorly when the data is highly skewed or has overlapping classes. In this case, the LDA model achieved an accuracy of 62.14%, which is significantly lower than the SVM and decision tree models. This could be due to the fact that the yalefaces dataset has high within-class variance and low between-class variance, making it difficult for the LDA model to find a good separation boundary.